<a href="https://colab.research.google.com/github/JHyunjun/PINN/blob/main/PINN_ECM_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Research start date : 23.08.23
# Project name : Deep learning-equivalent circuit model integrated battery system modeling

#%pip install --upgrade tensorflow

In [ ]:
# Importing
#%pip install tensorflow-probability

import numpy as np
import tensorflow as tf
from tensorflow import keras
from matplotlib import pyplot as plt
from tensorflow.keras.optimizers import Adam
import time
from keras.constraints import MinMaxNorm
from tensorflow.keras.layers import Dropout
from tensorflow.ekras.optimizers.schedules import ExponentialDecay
from mpl_toolkits.mplot3d import Axes3D
from sklearn.metrics import mean_squared_error

In [ ]:
# Data Load

xy_train = np.loadtxt('train dir/.', delimiter = ',', dtype = np.float32)
xy_test = np.loadtxt('test dir/.', delimiter = ',', dtype = np.float32)


In [ ]:
plt.figure(figsize = (10,6))
plt.plot(xy_train[:,-1], label = "Train Data", colr = "Red", linestyle = ":", alpha = 0.5)
plt.legend()
plt.grid(True)
plt.show()

plt.figure(figsize = (10,6))
plt.plot(xy_train[:,0], label = "Train Current", color = "Red", linestyle = ":", alpha = 0.5)
plt.legend()
plt.grid(True)
plt.show()

plt.figure(figsize = (10,6))
plt.plot(xy_train[:,2], label = "Train SOC", color = "Red", linestyle = ":", alpha = 0.5)
plt.legend()
plt.grid(True)
plt.show()


plt.figure(figsize = (10,6))
plt.plot(xy_test[:,-1], label = "Test Data", colr = "Red", linestyle = ":", alpha = 0.5)
plt.legend()
plt.grid(True)
plt.show()

plt.figure(figsize = (10,6))
plt.plot(xy_test[:,0], label = "Test Current", color = "Red", linestyle = ":", alpha = 0.5)
plt.legend()
plt.grid(True)
plt.show()

plt.figure(figsize = (10,6))
plt.plot(xy_test[:,2], label = "Test SOC", color = "Red", linestyle = ":", alpha = 0.5)
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Data Normalization

def origin_minmax(data) : #Mine
arr_max = np.zeros(data.shape[1])
arr_min = np.zeros(data.shape[1])

  for i in range(data.shape[1]) :
    max = np.max(data[:,i])
    min = np.min(data[:,i])
    arr_max[i] = max
    arr_min[i] = min
    pass
  return arr_max, arr_min

arr_max1, arr_min1 = origin_minmax(xy_train)
arr_max2, arr_min2 = origin_minmax(xy_test)
arr_total_max = []
arr_total_min = []

for i in range(0, len(arr_max1)) :
  if(arr_max1[i] > arr_max2[i]) :
    arr_total_max.append(arr_max1[i])
  else : arr_total_max.append(arr_max2[i])
  if(arr_min1[i] < arr_min2[i]) :
    arr_total_min.append(arr_min1[i])
  else : arr_total_min.append(arr_min2[i])

print(arr_total_max)
print(arr_total_min)

def MinMax_total_Scaler(data) :
  for i in range(0, data.shape[1]) :
    numerator = data[:,i] - arr_total_min[i]
    denominator = arr_total_max[i] - arr_total_min[i]
    data[:,i] = numerator / (denominator + 1e-7)
  return data

xy_train = MinMax_total_Scaler(xy_train)
xy_test = MinMax_total_Scaler(xy_test)

print(xy_train[:])
print(xy_test[:])

In [ ]:
# Data preprocessing
# Time table

x_train_data = xy_train[:, :-2]
time_axis_low = np.arange(28801)
time_axis_high = np.arange(len(xy)train - 28801)
time_axis = np.concatenate([time_axis_low, time_axis_high])
x_train_data = np.column_stack((time_axis, x_train_data))
x_train_data = np.around(x_train_data, 5)
y_train_data = xy_train[:,[-1]]
soc_train_data = xy_train[:,2]

print("Train")
print(x_train_data[-3:])
print(y_train_data[-3:])

x_test_Data = xy_test[:, :-2]
time_axis = np.arange(len(xy_test))
x_test_data = np.column_stack((time_axis, x_test_data))
x_test_data = np.around(x_test_data, 5)
y_test_data = xy_test[:,[-1]]
soc_test_data = xy_test[:,2]

print("Test")
print(x_test_data[-3:])
print(y_test_data[-3:])


In [ ]:
#Equivalent Circuit Modeling

# System Parameter
Series_connected = 180
Parallel_connected = 2
Amphere = 115

# SOC-OCV Table
soc_values = np.array([0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 78.6, 80, 85, 90, 95, 97, 100])
soc_normalizing = (soc_values - 0)/(100-0) # 0~1 Scailing
#soc_normalizing = ((soc_values - 0)/(100-0))*2-1 # -1~1 Scailing

ocv_values = np.array([3.256, 3.377, 3.419, 3.477, 3.528, 3.573, 3.603, 3.634, 3.660, 3.693, 3.740, 3.807, 3.865, 3.907, 3.947, 3.998, 4.016, 4.053, 4.088, 4.107, 4.136, 4.159, 4.224])
ocv_normalizing = (ocv_values - 3.256)/(4.224-3.256)

print(ocv_normalizing)
print(soc_normalizing)
interpolated_ocv = np.interp(31, soc_values,ocv_values)

def numpy_interpolation(x, soc_values, ocv_values):
    result = np.interp(x, soc_values, ocv_values)
    return result.astype(np.float32)

def tf_interpolation(x, soc_values, ocv_values):
    result = tf.py_function(numpy_interpolation, [x, soc_values, ocv_values], tf.float32)
    result.set_shape(x.get_shape())  # You may need to modify this to ensure shape compatibility
    return result

def custom_constraint(x, min_val, max_val):
    return tf.clip_by_value(x, min_val, max_val)

def custom_loss_with_constraints(y_true, y_pred, resistances):

    mse_loss = tf.keras.losses.mean_squared_error(y_true, y_pred)

    penalty = tf.maximum(0., resistances[:, 1] - resistances[:, 0]) + \
              tf.maximum(0., resistances[:, 2] - resistances[:, 1])

    lambda_penalty = 1.0

    return mse_loss + lambda_penalty * tf.reduce_mean(penalty)

In [ ]:
#ML

# Define the model

min_value = 0.0001 # 1mohm
max_value = 1 # 1000mohm
constraint = MinMaxNorm(min_value = min_value, max_value = max_value)
node = 128
dropout_rate = 0
lambda_penalty = 1

class ECM_ANN(keras.Model) :
    def __init__(self, soc_values, ocv_values) :
        super(ECM_ANN, self).__init__()
        self.dense1 = keras.layers.Dense(node,  kernel_initializer='he_normal')
        self.batch_norm1 = keras.layers.BatchNormalization()
        self.dropout1 = Dropout(dropout_rate)

        self.dense2 = keras.layers.Dense(node,  kernel_initializer='he_normal')
        self.batch_norm2 = keras.layers.BatchNormalization()
        self.dropout2 = Dropout(dropout_rate)

        self.dense3 = keras.layers.Dense(node,  kernel_initializer='he_normal')
        self.batch_norm3 = keras.layers.BatchNormalization()
        self.dropout3 = Dropout(dropout_rate)

        self.dense4 = keras.layers.Dense(node,  kernel_initializer='he_normal')
        self.batch_norm4 = keras.layers.BatchNormalization()
        self.dropout4 = Dropout(dropout_rate)

        self.dense5 = keras.layers.Dense(node,  kernel_initializer='he_normal')
        self.batch_norm5 = keras.layers.BatchNormalization()
        self.dropout5 = Dropout(dropout_rate)

        self.resistance = keras.layers.Dense(3,  kernel_initializer='he_normal')
        self.batch_norm_res = keras.layers.BatchNormalization()
        self.SOC_values = soc_normalizing
        self.OCV_values = ocv_normalizing

    def call(self, inputs) :

        #temp_and_soc = inputs[:, 1:]
        temp_and_soc = inputs[:, 2:]
        x = self.dense1(temp_and_soc)
        x = self.batch_norm1(x)
        x = tf.nn.relu(x)
        x = self.dropout1(x)

        x_shortcut_1 = x #Skip Connetion

        x = self.dense2(x)
        #x += x_shortcut_1
        x = self.batch_norm2(x)
        x = tf.nn.relu(x)
        x = self.dropout2(x)

        x_shortcut_2 = x

        x = self.dense3(x)
        #x += x_shortcut_2 #Skip Connection
        x = self.batch_norm3(x)
        x = tf.nn.relu(x)
        x = self.dropout3(x)

        x_shortcut_3 = x

        x = self.dense4(x)
        #x += x_shortcut_3
        x = self.batch_norm4(x)
        x = tf.nn.relu(x)
        x = self.dropout4(x)

        x_shortcut_4 = x

        x = self.dense5(x)
        #x += x_shortcut_4
        x = self.batch_norm5(x)
        x = tf.nn.relu(x)
        x = self.dropout5(x)

        resistance = self.resistance(x)
        resistance = self.batch_norm_res(resistance)
        resistance = tf.nn.relu(resistance)
        #resistance = tf.nn.sigmoid(resistance)
        #resistance = custom_constraint(resistance, min_value, max_value)  # Here, apply the custom constraint to the output.
        #tf.print("SOC : ",inputs[:,3]

        current = inputs[:,1]
        temp = inputs[:,2]
        soc = inputs[:,3]

        OCV_np = tf_interpolation(soc, self.SOC_values, self.OCV_values)
        OCV = tf.convert_to_tensor(OCV_np, dtype=tf.float32)
        #tf.print("OCV : ", OCV)
        R0, R1, R2 = resistance[:,0], resistance[:,1], resistance[:,2]

        # R0 > R1 > R2 Constraint
        penalty = tf.maximum(0., R1-R0)+ tf.maximum(0., R2-R1) + tf.maximum(0., R2-R0)
        epsilon = 1e-7
        penalty_zero = tf.maximum(0., epsilon-R0) + tf.maximum(0., epsilon-R1) + tf.maximum(0., 0.001 - epsilon-R2) # NonZero
        total_penalty = penalty + penalty_zero

        tau1 = 3
        tau2 = 10

        V_t0 = current * R0
        Q_t1 = current * R1
        Q_t2 = current * R2

        #terminal = OCV + V_t0 + Q_t1 + Q_t2
        terminal = OCV + V_t0 + Q_t1*(1 - tf.exp(-inputs[:,0]/tau1)) + Q_t2*(1-tf.exp(-inputs[:,0]/tau2))
        terminal = tf.nn.relu(terminal)
        #terminal = tf.nn.tanh(terminal)
        #terminal = tf.nn.leaky_relu(terminal, alpha = 0.3)
        #terminal_monitoring = terminal * (arr_total_max[-1] - arr_total_min[-1]) + arr_total_min[-1]
        #tf.print(terminal_monitoring)

        self.add_loss(lambda_penalty * tf.reduce_mean(total_penalty))

        return terminal

    def get_resistance(self, inputs):

        #temp_and_soc = inputs[:, 1:]
        temp_and_soc = inputs[:, 2:]
        x = self.dense1(temp_and_soc)
        x = self.batch_norm1(x)
        x = tf.nn.relu(x)
        x = self.dropout1(x)

        x_shortcut_1 = x

        x = self.dense2(x)
        #x += x_shortcut_1
        x = self.batch_norm2(x)
        x = tf.nn.relu(x)
        x = self.dropout2(x)

        x_shortcut_2 = x

        x = self.dense3(x)
        #x += x_shortcut_2 #Skip Connection
        x = self.batch_norm3(x)
        x = tf.nn.relu(x)
        x = self.dropout3(x)

        x_shortcut_3 = x

        x = self.dense4(x)
        #x += x_shortcut_3
        x = self.batch_norm4(x)
        x = tf.nn.relu(x)
        x = self.dropout4(x)

        x_shortcut_4 = x

        x = self.dense5(x)
        #x += x_shortcut_4
        x = self.batch_norm5(x)
        x = tf.nn.relu(x)
        x = self.dropout5(x)

        resistance = self.resistance(x)
        resistance = self.batch_norm_res(resistance)
        resistance = tf.nn.relu(resistance)
        #resistance = tf.nn.sigmoid(resistance)
        #resistance = custom_constraint(resistance, min_value, max_value)  # Here, apply the custom constraint to the output.

        return resistance

In [ ]:
# Model Compile

class LearningRateLogger(tf.keras.callbacks.Callback) :
  def on_epoch_begin(self, epoch, logs = None) :
    print(f"Epoch {epoch+1} - Learning rate : {self.model.optimizer.lr.numpy():.6f}")

start_time = time.time()

epoch = 300
#set learning rate decay
initial_learning_rate = 1e-3
lr_schedule = ExponentialDecay(initial_learning_rate, decay_steps = 1000, decay_rate = 0.995, staircase = True)

optimizer = tf.keras.optimizers.Adam(learning_rate = lr_schedule)
#optimizer = Adam(learning_rate = lr)
model = ECM_ANN(soc_normalizing, ocv_normalizing)
model.compile(optimizer = optimizer, loss = 'mse')
model.fit(x_train_data, y_train_data, batch_size = 256, epochs = epoch, callbacks [ LearningRateLogger()])

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Training time : {elapsed_time:.2f} seconds")

In [ ]:
# Validation with Train Data

predictions = model.predict(x_train_data)
y_train_data_2 = y_train_data * (arr_total_max[-1] - arr_total_min[-1]) + arr_total_min[-1]
predictions_2 = predictions * (arr_total_max[-1] - arr_total_min[-1]) + arr_total_min[-1]
y_train_data_2 = np.squeeze(y_train_data_2)

difference = y_train_data_2 - predictions_2

rmse = np.sqrt(mean_squared_error(y_train_data_2, predictions_2))
print(f"RMSE : {rmse:.4f}")

plt.figure(figsize = (15,8))
plt.plot(y_train_data_2, label = "True Values", color = "blue")
plt.plot(predictions_2, label = "Predictions", color = "red", linestyle = "--", alpha = 0.5)
plt.title("Model Training Performance")
plt.xlabel("Data Index")
plt.ylabel("Output Value")
plt.legend()
plt.grid(True)
plt.show()

plt.figure(figsize = (15,8))
plt.plot(difference, label = "Difference (Error)", color = "green", linestyle = ":", alpha = 0.5)
plt.ylim(-10,10)
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Plotting the resistance with train data

resistances = model.get_resistance(x_train_data).numpy()
R0 = resistances[:, 0]
R1 = resistances[:, 1]
R2 = resistances[:, 2]

plt.figure(figsize = (10,6))

plt.subplot(3, 1, 1)
plt.plot(R0, color = "blue", label = "R0 Values")
plt.title("[Train] R0 Values")
plt.grid(True)
plt.legend()

plt.subplot(3, 1, 2)
plt.plot(R1, color = "green" , label = "R1 Values")
plt.title("R1 Values")
plt.grid(True)
plt.legend()

plt.subplot(3, 1, 3)
plt.plot(R2, color = "red", label = "R2 Values")
plt.title("R2 Values")
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
# Plotting the resistance with train data

resistances = model.get_resistance(x_test_data).numpy()
R0 = resistances[:, 0]
R1 = resistances[:, 1]
R2 = resistances[:, 2]

plt.figure(figsize = (10,6))

plt.subplot(3, 1, 1)
plt.plot(R0, color = "blue", label = "R0 Values")
plt.title("[Test] R0 Values")
plt.grid(True)
plt.legend()

plt.subplot(3, 1, 2)
plt.plot(R1, color = "green" , label = "R1 Values")
plt.title("R1 Values")
plt.grid(True)
plt.legend()

plt.subplot(3, 1, 3)
plt.plot(R2, color = "red", label = "R2 Values")
plt.title("R2 Values")
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
# Resistance Plotting
# X_TRAIN_DATA : NUMBER/CURRENT/TEMP/SOC

soc = x_train_data[:, 3]
temperature = x_train_data[:, 2]

resistances = model.get_resistance(x_train_data).numpy()
R0 = resistances[:, 0]
R1 = resistances[:, 1]
R2 = resistances[:, 2]

fig = plt.figure(figsize=(18, 5))

ax1 = fig.add_subplot(131, projection='3d')
ax1.scatter(soc, temperature, R0, c=R0, cmap='viridis', s=10)
ax1.set_xlabel('SOC')
ax1.set_ylabel('Temperature')
ax1.set_zlabel('R0[ohm]')
ax1.set_title('R0 Distribution')

ax2 = fig.add_subplot(132, projection='3d')
ax2.scatter(soc, temperature, R1, c=R1, cmap='viridis', s=10)
ax2.set_xlabel('SOC')
ax2.set_ylabel('Temperature')
ax2.set_zlabel('R1[ohm]')
ax2.set_title('R1 Distribution')

ax3 = fig.add_subplot(133, projection='3d')
ax3.scatter(soc, temperature, R2, c=R2, cmap='viridis', s=10)
ax3.set_xlabel('SOC')
ax3.set_ylabel('Temperature')
ax3.set_zlabel('R2[ohm]')
ax3.set_title('R2 Distribution')

plt.tight_layout()
plt.show()

####################SOC , Current , Resistance
soc = x_train_data[:, 3]
current = x_train_data[:, 1]

fig = plt.figure(figsize=(18, 5))

ax1 = fig.add_subplot(131, projection='3d')
ax1.scatter(soc, current, R0, c=R0, cmap='viridis', s=10)
ax1.set_xlabel('SOC')
ax1.set_ylabel('Current')
ax1.set_zlabel('R0')
ax1.set_title('R0 Distribution')
ax1.view_init(elev=25, azim=115)

ax2 = fig.add_subplot(132, projection='3d')
ax2.scatter(soc, current, R1, c=R1, cmap='viridis', s=10)
ax2.set_xlabel('SOC')
ax2.set_ylabel('Current')
ax2.set_zlabel('R1')
ax2.set_title('R1 Distribution')
ax2.view_init(elev=25, azim=115)

ax3 = fig.add_subplot(133, projection='3d')
ax3.scatter(soc, current, R2, c=R2, cmap='viridis', s=10)
ax3.set_xlabel('SOC')
ax3.set_ylabel('Current')
ax3.set_zlabel('R2')
ax3.set_title('R2 Distribution')
ax3.view_init(elev=25, azim=115)  # Camera Angle

plt.tight_layout()
plt.show()

############################ 2D
soc = x_train_data[:, 3]
temp = x_train_data[:, 2]
current = x_train_data[:, 1]

fig, ax = plt.subplots(3, 3, figsize=(18, 15))

# R0, R1, R2 vs. SOC
ax[0, 0].scatter(soc, R0, c='r')
ax[0, 0].set_title('R0 vs. SOC')
ax[0, 0].set_xlabel('SOC')
ax[0, 0].set_ylabel('R0')

ax[0, 1].scatter(soc, R1, c='g')
ax[0, 1].set_title('R1 vs. SOC')
ax[0, 1].set_xlabel('SOC')
ax[0, 1].set_ylabel('R1')

ax[0, 2].scatter(soc, R2, c='b')
ax[0, 2].set_title('R2 vs. SOC')
ax[0, 2].set_xlabel('SOC')
ax[0, 2].set_ylabel('R2')

# R0, R1, R2 vs. Temperature
ax[1, 0].scatter(temp, R0, c='r')
ax[1, 0].set_title('R0 vs. Temperature')
ax[1, 0].set_xlabel('Temperature')
ax[1, 0].set_ylabel('R0')

ax[1, 1].scatter(temp, R1, c='g')
ax[1, 1].set_title('R1 vs. Temperature')
ax[1, 1].set_xlabel('Temperature')
ax[1, 1].set_ylabel('R1')

ax[1, 2].scatter(temp, R2, c='b')
ax[1, 2].set_title('R2 vs. Temperature')
ax[1, 2].set_xlabel('Temperature')
ax[1, 2].set_ylabel('R2')

# R0, R1, R2 vs. Current
ax[2, 0].scatter(current, R0, c='r')
ax[2, 0].set_title('R0 vs. Current')
ax[2, 0].set_xlabel('Current')
ax[2, 0].set_ylabel('R0')

ax[2, 1].scatter(current, R1, c='g')
ax[2, 1].set_title('R1 vs. Current')
ax[2, 1].set_xlabel('Current')
ax[2, 1].set_ylabel('R1')

ax[2, 2].scatter(current, R2, c='b')
ax[2, 2].set_title('R2 vs. Current')
ax[2, 2].set_xlabel('Current')
ax[2, 2].set_ylabel('R2')

plt.tight_layout()
plt.show()

In [ ]:
# Resistance Plotting
# X_TRAIN_DATA : NUMBER/CURRENT/TEMP/SOC

soc = x_test_data[:, 3]
temperature = x_test_data[:, 2]

resistances = model.get_resistance(x_test_data).numpy()
R0 = resistances[:, 0]
R1 = resistances[:, 1]
R2 = resistances[:, 2]

fig = plt.figure(figsize=(18, 5))

ax1 = fig.add_subplot(131, projection='3d')
ax1.scatter(soc, temperature, R0, c=R0, cmap='viridis', s=10)
ax1.set_xlabel('SOC')
ax1.set_ylabel('Temperature')
ax1.set_zlabel('R0[ohm]')
ax1.set_title('R0 Distribution')

ax2 = fig.add_subplot(132, projection='3d')
ax2.scatter(soc, temperature, R1, c=R1, cmap='viridis', s=10)
ax2.set_xlabel('SOC')
ax2.set_ylabel('Temperature')
ax2.set_zlabel('R1[ohm]')
ax2.set_title('R1 Distribution')

ax3 = fig.add_subplot(133, projection='3d')
ax3.scatter(soc, temperature, R2, c=R2, cmap='viridis', s=10)
ax3.set_xlabel('SOC')
ax3.set_ylabel('Temperature')
ax3.set_zlabel('R2[ohm]')
ax3.set_title('R2 Distribution')

plt.tight_layout()
plt.show()

####################SOC , Current , Resistance
soc = x_test_data[:, 3]
current = x_test_data[:, 1]

fig = plt.figure(figsize=(18, 5))

ax1 = fig.add_subplot(131, projection='3d')
ax1.scatter(soc, current, R0, c=R0, cmap='viridis', s=10)
ax1.set_xlabel('SOC')
ax1.set_ylabel('Current')
ax1.set_zlabel('R0')
ax1.set_title('R0 Distribution')
ax1.view_init(elev=25, azim=115)

ax2 = fig.add_subplot(132, projection='3d')
ax2.scatter(soc, current, R1, c=R1, cmap='viridis', s=10)
ax2.set_xlabel('SOC')
ax2.set_ylabel('Current')
ax2.set_zlabel('R1')
ax2.set_title('R1 Distribution')
ax2.view_init(elev=25, azim=115)

ax3 = fig.add_subplot(133, projection='3d')
ax3.scatter(soc, current, R2, c=R2, cmap='viridis', s=10)
ax3.set_xlabel('SOC')
ax3.set_ylabel('Current')
ax3.set_zlabel('R2')
ax3.set_title('R2 Distribution')
ax3.view_init(elev=25, azim=115)  # Camera Angle

plt.tight_layout()
plt.show()

############################ 2D
soc = x_test_data[:, 3]
temp = x_test_data[:, 2]
current = x_test_data[:, 1]

fig, ax = plt.subplots(3, 3, figsize=(18, 15))

# R0, R1, R2 vs. SOC
ax[0, 0].scatter(soc, R0, c='r')
ax[0, 0].set_title('R0 vs. SOC')
ax[0, 0].set_xlabel('SOC')
ax[0, 0].set_ylabel('R0')

ax[0, 1].scatter(soc, R1, c='g')
ax[0, 1].set_title('R1 vs. SOC')
ax[0, 1].set_xlabel('SOC')
ax[0, 1].set_ylabel('R1')

ax[0, 2].scatter(soc, R2, c='b')
ax[0, 2].set_title('R2 vs. SOC')
ax[0, 2].set_xlabel('SOC')
ax[0, 2].set_ylabel('R2')

# R0, R1, R2 vs. Temperature
ax[1, 0].scatter(temp, R0, c='r')
ax[1, 0].set_title('R0 vs. Temperature')
ax[1, 0].set_xlabel('Temperature')
ax[1, 0].set_ylabel('R0')

ax[1, 1].scatter(temp, R1, c='g')
ax[1, 1].set_title('R1 vs. Temperature')
ax[1, 1].set_xlabel('Temperature')
ax[1, 1].set_ylabel('R1')

ax[1, 2].scatter(temp, R2, c='b')
ax[1, 2].set_title('R2 vs. Temperature')
ax[1, 2].set_xlabel('Temperature')
ax[1, 2].set_ylabel('R2')

# R0, R1, R2 vs. Current
ax[2, 0].scatter(current, R0, c='r')
ax[2, 0].set_title('R0 vs. Current')
ax[2, 0].set_xlabel('Current')
ax[2, 0].set_ylabel('R0')

ax[2, 1].scatter(current, R1, c='g')
ax[2, 1].set_title('R1 vs. Current')
ax[2, 1].set_xlabel('Current')
ax[2, 1].set_ylabel('R1')

ax[2, 2].scatter(current, R2, c='b')
ax[2, 2].set_title('R2 vs. Current')
ax[2, 2].set_xlabel('Current')
ax[2, 2].set_ylabel('R2')

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

soc_values = np.unique(x_test_data[:,3])
avg_R0 = []
avg_R1 = []
avg_R2 = []

for soc in soc_values :
  soc_indices = np.where(x_test_data[:,3] == soc)
  soc_R0 = R0[soc_indices]
  soc_R1 = R1[soc_indices]
  soc_R2 = R2[soc_indices]

  avg_R0.append(np.mean(soc_R0))
  avg_R1.append(np.mean(soc_R1))
  avg_R2.append(np.mean(soc_R2))

plt.figure(figsize = (12,10))

plt.subplot(3, 1, 1)
plt.plot(soc_values, avg_R0, color = "blue", marker = '.', label = "Average R0 Values", linestyle = 'None')
plt.title("Average R0 Values")
plt.xlabel("SOC")
plt.ylabel("Resistance")
plt.gird(True)
plt.legend()

plt.subplot(3, 1, 2)
plt.plot(soc_values, avg_R1, color = "blue", marker = '.', label = "Average R1 Values", linestyle = 'None')
plt.title("Average R1 Values")
plt.xlabel("SOC")
plt.ylabel("Resistance")
plt.gird(True)
plt.legend()

plt.subplot(3, 1, 3)
plt.plot(soc_values, avg_R2, color = "blue", marker = '.', label = "Average R2 Values", linestyle = 'None')
plt.title("Average R2 Values")
plt.xlabel("SOC")
plt.ylabel("Resistance")
plt.gird(True)
plt.legend()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

temp_values = np.unique(x_test_data[:,2])
avg_R0 = []
avg_R1 = []
avg_R2 = []

for temp in temp_values :
  temp_indices = np.where(x_test_data[:,2] == temp)
  temp_R0 = R0[temp_indices]
  temp_R1 = R1[temp_indices]
  temp_R2 = R2[temp_indices]

  avg_R0.append(np.mean(temp_R0))
  avg_R1.append(np.mean(temp_R1))
  avg_R2.append(np.mean(temp_R2))

plt.figure(figsize = (12,10))

plt.subplot(3, 1, 1)
plt.plot(temp_values, avg_R0, color = "blue", marker = '.', label = "Average R0 Values", linestyle = 'None')
plt.title("Average R0 Values")
plt.xlabel("Temperature")
plt.ylabel("Resistance")
plt.gird(True)
plt.legend()

plt.subplot(3, 1, 2)
plt.plot(temp_values, avg_R1, color = "blue", marker = '.', label = "Average R1 Values", linestyle = 'None')
plt.title("Average R1 Values")
plt.xlabel("Temperature")
plt.ylabel("Resistance")
plt.gird(True)
plt.legend()

plt.subplot(3, 1, 3)
plt.plot(temp_values, avg_R2, color = "blue", marker = '.', label = "Average R2 Values", linestyle = 'None')
plt.title("Average R2 Values")
plt.xlabel("Temperature")
plt.ylabel("Resistance")
plt.gird(True)
plt.legend()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

current_values = np.unique(x_test_data[:,1])
avg_R0 = []
avg_R1 = []
avg_R2 = []

for current in current_values :
  current_indices = np.where(x_test_data[:,1] == current)
  current_R0 = R0[current_indices]
  current_R1 = R1[current_indices]
  current_R2 = R2[current_indices]

  avg_R0.append(np.mean(current_R0))
  avg_R1.append(np.mean(current_R1))
  avg_R2.append(np.mean(current_R2))

plt.figure(figsize = (12,10))

plt.subplot(3, 1, 1)
plt.plot(current_values, avg_R0, color = "blue", marker = '.', label = "Average R0 Values", linestyle = 'None')
plt.title("Average R0 Values")
plt.xlabel("Current")
plt.ylabel("Resistance")
plt.gird(True)
plt.legend()

plt.subplot(3, 1, 2)
plt.plot(current_values, avg_R1, color = "blue", marker = '.', label = "Average R1 Values", linestyle = 'None')
plt.title("Average R1 Values")
plt.xlabel("Current")
plt.ylabel("Resistance")
plt.gird(True)
plt.legend()

plt.subplot(3, 1, 3)
plt.plot(current_values, avg_R2, color = "blue", marker = '.', label = "Average R2 Values", linestyle = 'None')
plt.title("Average R2 Values")
plt.xlabel("Current")
plt.ylabel("Resistance")
plt.gird(True)
plt.legend()